In [1]:
import caffe
import numpy as np
import matplotlib.pyplot as plt
import csv
import time
from IPython.display import clear_output
%matplotlib inline

import copy #targetpoint

# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a (potentially misleading) color heatmap

caffe.set_mode_gpu()
caffe.set_device(0)

mu = np.load('//home//student401//anaconda3//pkgs//caffe-1.0-py36hdb41b07_3//lib//python3.6//site-packages//caffe//imagenet//ilsvrc_2012_mean.npy')
mu = mu.mean(1).mean(1)  # average over pixels to obtain the mean (BGR) pixel values
mu

array([104.00698793, 116.66876762, 122.67891434])

In [2]:
class CaffeNet():
    def __init__(self, net_def_path, net_weight_path):
        self.__def_path = net_def_path
        self.__weights_path = net_weight_path
        
        self.__net = None
        self.__transformer = None
        
        self.__init_net_transformer()
    
    
    def get_inner_net(self):
        return self.__net
        
        
    def __init_net_transformer(self):
        net = caffe.Net(self.__def_path, self.__weights_path, caffe.TEST)     

        #transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
        #transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
        #transformer.set_mean('data', mu)            # subtract the dataset-mean value in each channel
        #transformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
        #transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR
        
        self.__net = net
        #self.__transformer = transformer
        
        
    # FUNCTION DOESN'T USE SELF PARAMETER
        
    def __show_progress(percent, message = 'Progress'):
        clear_output(1)
        hash_count = round(50 * percent)
        hash_string = ''
        
        for i in range(hash_count):
            hash_string += '#'
        
        for i in range(50 - hash_count):
            hash_string += '-'
            
        print(message+': '+hash_string)
        
        
    def load_multiple_images(image_path_arr):
        """
        Load images to python list.
        Then return this list.
        """
        image_arr = []
        i = 0
        count = len(image_path_arr)
        

        for image_path in image_path_arr:
            image_arr.append(caffe.io.load_image(image_path, color=False))
            i += 1
            CaffeNet.__show_progress(i/count, 'Loading images')

        return image_arr
    
    def convert_full_data_to_simple(answer_full_data_arr_l):
        """
        Convert full data to standart (label, confidence) list
        """
        answer_arr_l = []

        for layer_data_l in answer_full_data_arr_l:
            data_tuple_l = (layer_data_l.argmax(), layer_data_l.max())
            answer_arr_l.append(data_tuple_l)

        return answer_arr_l
    
    
    # SIMPLE DATA FUNCTIONS
    
        
    def predict(self, image_path):
        """
        Predict single image.
        Function returns a tuple - (confidence, label)
        """
        image = caffe.io.load_image(image_path)

        transformed_image = self.__transformer.preprocess('data',image)
        self.__net.blobs['data'].data[...] = transformed_image
        output = self.__net.forward()
        output_prob = output['prob'][0]
        return (output_prob.argmax(), output_prob.max())
     
        
    def __predict_transfromed_image(self, transformed_image):
        """
        Predict and return (label, confidence)
        """
        self.__net.blobs['data'].data[...] = transformed_image
        output = self.__net.forward()
        output_prob = output['prob'][0]
        return (output_prob.argmax(), output_prob.max())
    
        
    def __transform_image_arr(self, image_arr):
        """
        Transform each image in array and return an array of transformed images
        """
        transformed_image_arr_t = []
        i = 0
        count = len(image_arr)
        
        for image_t in image_arr:
            transformed_image_arr_t.append(self.__transformer.preprocess('data',image_t))
            i += 1
            CaffeNet.__show_progress(i/count, 'Transforming')
            
        return transformed_image_arr_t;
    
        
    def predict_multi(self, image_path_arr, image_arr = 0):
        """
        Predict multiple images.
        Function returns a list of tuples - (label, confidence)
        """
        if image_arr == 0:
            image_arr = CaffeNet.load_multiple_images(image_path_arr)
            
        transformed_image_arr_r = image_arr

        answers = []
        i = 0
        count = len(image_path_arr)

        for transformed_image_r in transformed_image_arr_r:
            answers.append(self.__predict_transfromed_image(transformed_image_r))
            i += 1
            CaffeNet.__show_progress(i/count, 'Predicting')
            
        return answers
    
    
    # FULL DATA FUNCTIONS
    
    
    def predict_multi_full_data(self, image_path_arr, image_arr = 0):
        """
        Predict multiple images.
        Function returns a list of lists which contain last layer values
        """
        
        if image_arr == 0:
            input('F')
            image_arr = CaffeNet.load_multiple_images(image_path_arr)

        transformed_image_arr_p = image_arr
        
        answer_arr_p = []
        
        i = 0
        count = len(image_arr)
        
        for transformed_image_p in transformed_image_arr_p:
            answer_arr_p.append(copy.copy(self.__predict_transfromed_image_full_data(transformed_image_p))) #targetpoint
            i += 1
            CaffeNet.__show_progress(i/count, 'Predicting')
            
        return answer_arr_p
    
    def __predict_transfromed_image_full_data(self, transformed_image_l):
        """
        Predict and return values of the last layer
        """
        self.__net.blobs['data'].data[...] = transformed_image_l
        output = self.__net.forward()
        output_prob = output['prob'][0]
        return output_prob
    
    
    # OTHER
    
    
    def get_last_layer_size(self):
        output = self.__net.forward()
        output_prob = output['prob'][0]
        return len(output_prob)
    

In [3]:
vgg10_proto = 'proto/vgg10_test.prototxt'
vgg10_weigts = 'vgg10_100k_lr1e-1_iter_100000.caffemodel'


In [11]:
"FUNCTION FOR GETTING ARRAY OF LINKS TO IMAGES"
import os

def get_links_to_img(path):
    """Return array of links to images"""
    link_arr = []
    for root, dirs, files in os.walk(top = path):
       for filename in files:
        link_arr.append(path+'/'+filename)
    return link_arr

In [12]:


TestImagesSet = '/mnt/data/dendriticDataSet'
image_path_arr = get_links_to_img(TestImagesSet)
print(len(image_path_arr))

1043600


In [13]:
image_path_arr[0]

'/mnt/data/dendriticDataSet/Crystal_F32_IMC=0.11_IDC=0.3_IEC=0.06_Step=76111_ID=4_crop134_.png'

In [4]:
vgg10 = CaffeNet(vgg10_proto, vgg10_weigts)

In [29]:
vgg10.predict_multi(image_path_arr[0:100])

Loading images: ##################################################


ValueError: could not broadcast input array from shape (224,224,3) into shape (32,1,224,224)

In [40]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
plt.switch_backend('agg')
# For loading dendrite images
from scipy import misc


def get_data():
    df = pd.read_csv('/home/student401/study/data_sets/mnist/train.csv')
    data = df.values

    X = data[:, 1:]
    Y = data[:, 0]

    return X, Y


def one_hot_encoding(Y):
    N = len(Y)
    D = len(set(Y))
    new_Y = np.zeros((N, D))

    for i in range(N):
        new_Y[i, Y[i]] = 1

    return new_Y

# For denrites
def one_hot_encoding_d(Y):
    N = len(Y)
    D = len(set(Y))
    new_Y = np.zeros((N, D))

    for i in range(N):
        new_Y[i, Y[i] - 1] = 1

    return new_Y


def get_preprocessed_data():
    X, Y = get_data()
    X = X / 255
    Y = one_hot_encoding(Y)
    X, Y = shuffle(X, Y)
    return X, Y


def error_rate(Y, T):
    return np.mean(Y != T)


def create_graph(value, name, y_label, x_label):
    fig = plt.figure()
    fig.set_size_inches(16, 9)

    axes = fig.add_axes([0.1, 0.1, 0.8, 0.8])
    axes.plot(value)
    axes.set_ylabel(y_label)
    axes.set_xlabel(x_label)

    fig.savefig(name)


def rearrange(X):
    # input is (N, 784)
    # output is (N, 28, 28)
    new_X = np.zeros((X.shape[0], 1, 28, 28))
    for pixels_row, new_X_mat in zip(X, new_X):
        for j in range(28):
            new_X_mat[0, j] += pixels_row[j*28:(j+1)*28]

    return new_X.astype(np.float32)


def rearrange_tf(X):
    # input is (N, 784)
    # output is (N, 28, 28, 1)
    new_X = np.zeros((X.shape[0], 28, 28, 1))
    for j in range(28):
        new_X[:,j,:,0] += X[:,j*28:(j+1)*28]

    return new_X

def rearrange_dendrites_tf(X):
    # input is (N, 224, 224)
    # output is (N, 224, 224, 1)
    new_X = np.zeros((X.shape[0], 224, 224, 1))
    new_X[:, :, :, 0] = X[:, :, :]

    return new_X

def rearrange_dendrites_caffe(X):
    # input is (N, 224, 224)
    # output is (N, 1, 224, 224)
    new_X = np.zeros((X.shape[0], 1, 224, 224))
    new_X[:, 0, :, :] = X[:, :, :]

    return new_X


def get_preprocessed_image_data():
    X, Y = get_preprocessed_data()
    X = rearrange(X)
    return X, Y


def get_preprocessed_image_data_tf():
    X, Y = get_preprocessed_data()
    X = rearrange_tf(X)
    return X, Y


def load_train_dendrit_data():
    prepath = '/home/rustam2/nvcaffeExample/NvCaffeDendriticStructureClassifier/ResNetUsage/'
    path_to_train_txt = '/home/student401/study/data_sets/dendrits/train.txt'

    file = open(path_to_train_txt)
    text = file.read()
    text = text.split('\n')[:-1]

    image_paths = []
    classes = []
    for pair in text:
        path, cl = pair.split(' ')
        image_paths.append(path)
        classes.append(cl)

    for i in range(len(image_paths)):
        image_paths[i] = prepath + image_paths[i]

    images = [misc.imread(path) for path in image_paths]
    images = np.asarray(images)

    # Convert classes to numbers since they are strings
    Y = []
    for i in range(len(classes)):
        Y.append(int(classes[i]))

    return images, Y

def load_test_dendrit_data():
    #prepath = '/home/rustam2/nvcaffeExample/NvCaffeDendriticStructureClassifier/ResNetUsage/'
    path_to_train_txt = '/home/rustam2/nvcaffeExample/NvCaffeDendriticStructureClassifier/ResNetUsage/resources/train.txt'

    file = open(path_to_train_txt)
    text = file.read()
    text = text.split('\n')[:-1]

    image_paths = []
    classes = []
    for pair in text[:5000]:
        path, cl = pair.split(' ')
        image_paths.append(path)
        classes.append(cl)

    for i in range(len(image_paths)):
        image_paths[i] = image_paths[i]

    images = [misc.imread(path) for path in image_paths]
    images = np.asarray(images)

    # Convert classes to numbers since they are strings
    Y = []
    for i in range(len(classes)):
        Y.append(int(classes[i]))

    return images, Y

def get_dendrite_preprocessed_train_data_tf():
    # X shape is 224x224
    X, Y = load_train_dendrit_data()

    # Preprocess X
    X = X / 255
    X = rearrange_dendrites_tf(X)

    # Convert to np array since Y is vanilla python list
    Y = np.array(Y)
    Y = one_hot_encoding_d(Y)
    return X, Y

def get_dendrite_preprocessed_test_data_tf():
    # X shape is 224x224
    X, Y = load_test_dendrit_data()

    # Preprocess X
    X = X / 255
    X = rearrange_dendrites_tf(X)

    # Convert to np array since Y is vanilla python list
    Y = np.array(Y)
    Y = one_hot_encoding_d(Y)
    return X, Y

def get_dendrite_preprocessed_test_data_caffe():
    # X shape is 224x224
    X, Y = load_test_dendrit_data()

    # Preprocess X
    X = X / 255
    X = rearrange_dendrites_caffe(X)

    # Convert to np array since Y is vanilla python list
    Y = np.array(Y)
    Y = one_hot_encoding_d(Y)
    return X, Y

In [41]:
X, Y = get_dendrite_preprocessed_test_data_caffe()

/home/rustam2/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:161: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


In [42]:
X.shape

(5000, 1, 224, 224)

In [101]:
def predict(caffe_net, images):
    batch_sz = 32
    n_batches = images.shape[0] // batch_sz
    net = caffe_net.get_inner_net()
    predictions = []
    for i in range(n_batches):
        Xbatch = images[i*batch_sz:(i+1)*batch_sz]
        net.blobs['data'].data[...] = Xbatch
        output = net.forward()
        #output = net.blobs['score'].data[...]
        for prob in output['score'][::-1]:
            predictions.append(prob.argmax())
        
        
    return predictions

In [102]:
def classification_rate(Y, T):
    return np.mean(Y == T)

predictions2 = predict(vgg10, X[:1000])

In [100]:
np.array([[1],[2]])[::-1]

array([[2],
       [1]])

In [77]:
np.argmax(Y[:32] ,axis=1)

array([ 4,  6,  0,  6,  4,  0,  5,  7,  8,  8, 10,  2, 10,  0,  9,  2,  2,
        3,  8,  8,  9,  7,  2,  6,  0,  0, 10,  5,  4,  7,  4,  6])

In [103]:
classification_rate(np.array(predictions2), np.argmax(Y[:992], axis=1))

0.09979838709677419

In [80]:
[(k, v.data.shape) for k, v in vgg10.get_inner_net().blobs.items()]
#predictions = predict(vgg10, X)

[('data', (32, 1, 224, 224)),
 ('label', (32,)),
 ('label_data_1_split_0', (32,)),
 ('label_data_1_split_1', (32,)),
 ('label_data_1_split_2', (32,)),
 ('conv1_1', (32, 64, 224, 224)),
 ('conv1_2', (32, 64, 224, 224)),
 ('pool1', (32, 64, 112, 112)),
 ('conv2_1', (32, 128, 112, 112)),
 ('conv2_2', (32, 128, 112, 112)),
 ('pool2', (32, 128, 56, 56)),
 ('conv3_1', (32, 256, 56, 56)),
 ('conv3_2', (32, 256, 56, 56)),
 ('conv3_3', (32, 256, 56, 56)),
 ('pool3', (32, 256, 28, 28)),
 ('conv4_1', (32, 512, 28, 28)),
 ('conv4_2', (32, 512, 28, 28)),
 ('conv4_3', (32, 512, 28, 28)),
 ('pool4', (32, 512, 14, 14)),
 ('conv5_1', (32, 512, 14, 14)),
 ('conv5_2', (32, 512, 14, 14)),
 ('conv5_3', (32, 512, 14, 14)),
 ('pool5', (32, 512, 7, 7)),
 ('fc1', (32, 4096)),
 ('fc2', (32, 2048)),
 ('score', (32, 11)),
 ('score_score_0_split_0', (32, 11)),
 ('score_score_0_split_1', (32, 11)),
 ('score_score_0_split_2', (32, 11)),
 ('loss', ()),
 ('accuracy_top1', ()),
 ('accuracy_top5', ())]

CREATE DATAFRAMES WITH ANSWERS OF EACH NETWORK

In [7]:
columns = ['GOOGLENET','ALEXNET','VGG16','RESNET']
rows = image_path_arr

df = pd.DataFrame(np.random.randn(len(rows),len(columns)), rows, columns)
df.to_csv('confidence.csv')
df.to_csv('labels.csv')

df_confidence = pd.DataFrame.from_csv('confidence.csv')
df_labels = pd.DataFrame.from_csv('labels.csv')

/home/student401/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  
/home/student401/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  if __name__ == '__main__':


In [8]:
def df_each_neuron_data(net_tuple, columns):
    net = CaffeNet(net_tuple[0], net_tuple[1])
    last_layer_size = net.get_last_layer_size()
    
    rows = np.array(range(last_layer_size))
    
    df = pd.DataFrame(np.random.randn(len(rows),len(columns)), rows, columns)
    return df

In [9]:
df_googlenet = df_each_neuron_data(googlenet, image_path_arr)
df_alexnet = df_each_neuron_data(alexnet, image_path_arr)
df_vgg16 = df_each_neuron_data(vgg16, image_path_arr)
df_resnet = df_each_neuron_data(resnet152, image_path_arr)

SAVE RESULTS FROM EACH NET

In [10]:
def save_prediction_to_df(df, column_name, answer_arr, mark0_conf1 = 0):
    """
    This function save prediction results to dataframe df.
    mark0_conf1: 0 - save label, 1 - save confidence
    """
    
    for i in range(len(df[column_name])):
        df[column_name][i] = answer_arr[i][mark0_conf1]
        
def save_prediction_full_data_to_df(df, answer_full_data_arr, image_path_arr):
    """
    This function save full data from a certain net to dataframe
    """
    for i in range(len(df_googlenet.iloc[0])):
        for j in range(len(df[image_path_arr[0]])):
            df[image_path_arr[i]][j] = answer_full_data_arr[i][j]

In [11]:
# image_path_arr declared above
image_path_arr
columns = ['GOOGLENET','ALEXNET','VGG16','RESNET']
net_arr = [googlenet, alexnet, vgg16, resnet152]
df_full_data_arr = [df_googlenet, df_alexnet, df_vgg16, df_resnet]
loaded_image_arr = CaffeNet.load_multiple_images(image_path_arr=image_path_arr)


Loading images: ##################################################


In [12]:

for i in range(4):
    # Load and compile net
    net = CaffeNet(net_arr[i][0], net_arr[i][1])
    # Get answers
    answer_arr_full_data = net.predict_multi_full_data(image_path_arr, loaded_image_arr)
    answer_arr = CaffeNet.convert_full_data_to_simple(answer_arr_full_data)
    # Save simple results
    save_prediction_to_df(df_labels, columns[i], answer_arr, 0)
    save_prediction_to_df(df_confidence, columns[i], answer_arr, 1)
    # Save full data results
    save_prediction_full_data_to_df(df_full_data_arr[i], answer_arr_full_data, image_path_arr)

Predicting: ##################################################


In [13]:
df_confidence.to_csv('confidence.csv')
df_labels.to_csv('labels.csv')

df_googlenet.to_csv('googlenet.csv')
df_alexnet.to_csv('alexnet.csv')
df_vgg16.to_csv('vgg16.csv')
df_resnet.to_csv('resnet.csv')

In [14]:
df_labels.head()

,GOOGLENET,ALEXNET,VGG16,RESNET
//home//student401//neuralnets//DataSet//val_256//Places365_val_00020611.jpg,145.0,197.0,145.0,145.0
//home//student401//neuralnets//DataSet//val_256//Places365_val_00027736.jpg,359.0,359.0,359.0,359.0
//home//student401//neuralnets//DataSet//val_256//Places365_val_00010687.jpg,42.0,42.0,42.0,42.0
//home//student401//neuralnets//DataSet//val_256//Places365_val_00018876.jpg,195.0,195.0,195.0,195.0
//home//student401//neuralnets//DataSet//val_256//Places365_val_00004086.jpg,215.0,121.0,215.0,215.0


In [15]:
df_confidence.head()

,GOOGLENET,ALEXNET,VGG16,RESNET
//home//student401//neuralnets//DataSet//val_256//Places365_val_00020611.jpg,0.856080,0.350724,0.806077,0.529477
//home//student401//neuralnets//DataSet//val_256//Places365_val_00027736.jpg,0.444585,0.491834,0.529767,0.843238
//home//student401//neuralnets//DataSet//val_256//Places365_val_00010687.jpg,0.667495,0.547526,0.838639,0.926617
//home//student401//neuralnets//DataSet//val_256//Places365_val_00018876.jpg,0.972118,0.941433,0.976265,0.997452
//home//student401//neuralnets//DataSet//val_256//Places365_val_00004086.jpg,0.398947,0.294820,0.764184,0.393228


In [16]:
df_googlenet.head()

,//home//student401//neuralnets//DataSet//val_256//Places365_val_00020611.jpg,//home//student401//neuralnets//DataSet//val_256//Places365_val_00027736.jpg,//home//student401//neuralnets//DataSet//val_256//Places365_val_00010687.jpg,//home//student401//neuralnets//DataSet//val_256//Places365_val_00018876.jpg,//home//student401//neuralnets//DataSet//val_256//Places365_val_00004086.jpg,//home//student401//neuralnets//DataSet//val_256//Places365_val_00020247.jpg,//home//student401//neuralnets//DataSet//val_256//Places365_val_00032581.jpg,//home//student401//neuralnets//DataSet//val_256//Places365_val_00003318.jpg,//home//student401//neuralnets//DataSet//val_256//Places365_val_00034661.jpg,//home//student401//neuralnets//DataSet//val_256//Places365_val_00018961.jpg,...,//home//student401//neuralnets//DataSet//val_256//Places365_val_00014336.jpg,//home//student401//neuralnets//DataSet//val_256//Places365_val_00008776.jpg,//home//student401//neuralnets//DataSet//val_256//Places365_val_00006841.jpg,//home//student401//neuralnets//DataSet//val_256//Places365_val_00015991.jpg,//home//student401//neuralnets//DataSet//val_256//Places365_val_00025377.jpg,//home//student401//neuralnets//DataSet//val_256//Places365_val_00021812.jpg,//home//student401//neuralnets//DataSet//val_256//Places365_val_00005874.jpg,//home//student401//neuralnets//DataSet//val_256//Places365_val_00006424.jpg,//home//student401//neuralnets//DataSet//val_256//Places365_val_00027391.jpg,//home//student401//neuralnets//DataSet//val_256//Places365_val_00001122.jpg
0,2.159526e-07,1.543440e-04,7.339061e-06,8.759868e-09,0.000003,2.162869e-08,3.721017e-06,0.000007,1.997119e-07,7.926048e-07,...,0.000049,0.000002,7.761012e-07,1.223886e-06,1.539239e-05,4.986495e-07,0.000006,4.574176e-07,0.000487,9.738069e-08
1,6.975498e-09,1.205092e-06,3.664356e-08,4.173852e-07,0.000015,2.951170e-08,4.130065e-07,0.000005,2.866622e-08,2.116008e-05,...,0.000025,0.000005,1.981672e-08,1.735944e-05,5.106659e-08,1.045868e-06,0.000013,1.602975e-07,0.000096,1.248269e-06
2,1.099129e-07,1.720226e-06,7.896384e-07,8.642601e-07,0.000019,4.284949e-09,2.011322e-06,0.000003,8.860980e-06,1.211861e-04,...,0.000036,0.000002,3.958298e-08,6.013190e-05,8.487540e-06,8.851688e-08,0.000078,1.128363e-07,0.000502,1.677633e-02
3,4.276428e-07,2.392427e-07,2.638800e-07,2.374833e-05,0.003146,8.817320e-07,1.485833e-05,0.000003,5.312104e-08,1.356587e-03,...,0.000001,0.000002,1.732927e-07,6.878187e-06,2.751497e-07,2.705490e-05,0.000116,6.691239e-06,0.000002,1.036851e-06
4,3.075771e-06,1.531285e-06,2.727855e-07,9.034971e-09,0.000004,2.364391e-07,6.416658e-07,0.000001,1.343919e-05,4.453065e-07,...,0.000009,0.000002,5.080334e-07,8.908675e-07,8.741235e-08,1.012164e-04,0.000165,1.176754e-04,0.000115,2.573156e-06


TEST CELLS

In [ ]:
plt.imshow(loaded_image_arr[1])
print(len(loaded_image_arr))

36500


In [ ]:
alexnet = CaffeNet(alexnet_def, alexnet_w)
# HERE IS A PROBLEM
answer_arr_full_data = alexnet.predict_multi_full_data(image_path_arr, loaded_image_arr)
#print((answer_arr_full_data_tt))
answer_arr = CaffeNet.convert_full_data_to_simple(answer_arr_full_data_tt)
print((answer_arr))

#print(alexnet.convert_full_data_to_simple(alexnet.predict_multi_full_data(image_path_arr, loaded_image_arr[:-98])))

In [ ]:
answer_arr_tt[2]

In [ ]:
start_time = time.time()
for i in range(len(answer_arr)):
    start_time = time.time()
    for j in range(len(df_alexnet[image_path_arr[0]])):
        df_alexnet[image_path_arr[i]][j] = answer_arr_full_data[i][j]

In [ ]:
df_alexnet.head()

In [ ]:
answer_arr = CaffeNet.convert_full_data_to_simple(answer_arr_full_data_arr)

In [ ]:
answer_arr